In [286]:
import pandas as pd
import numpy as np

df = pd.read_csv(r"../data/class_german_credit.csv", engine="pyarrow")

In [287]:
df

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,67,male,2,own,None,little,1169,6,radio/TV,good
1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,49,male,1,own,little,None,2096,12,education,good
3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,53,male,2,free,little,little,4870,24,car,bad
...,...,...,...,...,...,...,...,...,...,...
995,31,female,1,own,little,None,1736,12,furniture/equipment,good
996,40,male,3,own,little,little,3857,30,car,good
997,38,male,2,own,little,None,804,12,radio/TV,good
998,23,male,2,free,little,little,1845,45,radio/TV,bad


# Pré-processamento dos dados

### Limpeza de dados

#### Remoção de linhas com missing data

In [288]:
mapeamento = {'little': 0, 'moderate': 1, 'quite rich':3, 'rich':2 }

df['Checking account'] = df['Checking account'].map(mapeamento)   

In [289]:
# aqui eu usei o knn para completar valores missing do checking account, apenas utilizei valores numericos

from sklearn.impute import KNNImputer
coluna_alvo = 'Checking account'
colunas_numericas = df.select_dtypes(include='number').columns
colunas_auxiliares = colunas_numericas.drop(coluna_alvo)


df_aux = df.dropna(subset=colunas_auxiliares)

imputer = KNNImputer(n_neighbors=3)
df[coluna_alvo] = imputer.fit_transform(df[[coluna_alvo] + list(colunas_auxiliares)])[:, 0]


In [290]:
#df.drop('Checking account',axis='columns', inplace=True)

# o numero de NAs dessa coluna é muito alto, perco quase metade do dataset quando mantenho ela
# após balancear a base o KNN gerou bons resultados come essa coluna

In [291]:
df.dropna(axis=0,inplace= True)

In [292]:
df

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
1,22,female,2,own,little,1.000000,5951,48,radio/TV,bad
2,49,male,1,own,little,1.666667,2096,12,education,good
3,45,male,2,free,little,0.000000,7882,42,furniture/equipment,good
4,53,male,2,free,little,0.000000,4870,24,car,bad
6,53,male,2,own,quite rich,0.666667,2835,24,furniture/equipment,good
...,...,...,...,...,...,...,...,...,...,...
995,31,female,1,own,little,0.333333,1736,12,furniture/equipment,good
996,40,male,3,own,little,0.000000,3857,30,car,good
997,38,male,2,own,little,0.333333,804,12,radio/TV,good
998,23,male,2,free,little,0.000000,1845,45,radio/TV,bad


#### Remoção de outliers

In [293]:
#aqui eu testei eliminar os outliers baseado no boxplot da analise, melhorou os resultados

Q1 = df['Credit amount'].quantile(0.25) # Primeiro quartil
Q2 = df['Credit amount'].quantile(0.50) # Segundo quartil 
Q3 = df['Credit amount'].quantile(0.75) # Terceiro quartil
print(f"Q1: {Q1}")
print(f"Q2: {Q2}")
print(f"Q3: {Q3}\n")
print(df['Credit amount'].median())

df = df[df['Credit amount'] < Q3 + 1.5 * (Q3-Q1)]

Q1: 1355.0
Q2: 2241.0
Q3: 3850.0

2241.0


In [294]:
#aqui eu testei eliminar os outliers baseado no boxplot da analise, não melhorou o resultado

# q1 = df['Duration'].quantile(0.25) # primeiro quartil
# q2 = df['Duration'].quantile(0.50) # segundo quartil 
# q3 = df['Duration'].quantile(0.75) # terceiro quartil
# print(f"q1: {q1}")
# print(f"q2: {q2}")
# print(f"q3: {q3}\n")
# print(df['Duration'].median())

# df = df[df['Duration'] < q3 + 1.5 * (q3-q1)]


In [295]:
df = df[df['Job'] != 0]
#resultados melhoram apóes dropar instancias com job '0'

### Normalização e Trasformação

#### Normalizaação


In [296]:
# #utilizando função do sklearn para aplicar o zscore
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# df['Age'] = scaler.fit_transform(df[['Age']])
# df['Duration'] = scaler.fit_transform(df[['Duration']])
# df['Credit amount'] = scaler.fit_transform(df[['Credit amount']])

In [297]:
#tanto a min max quanto a normalização zscore nao alteraram os resultados da decision tree
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df['Age'] = scaler.fit_transform(df[['Age']])
df['Duration'] = scaler.fit_transform(df[['Duration']])
df['Credit amount'] = scaler.fit_transform(df[['Credit amount']])

#### Encoding Ordinal

In [298]:
#agrupando quite rich e rich como a mesma classe devido ao baixo numero de instancias
#mapeamento = {'little': 0, 'moderate': 1, 'quite rich':2, 'rich':2 }

mapeamento = {'little': 0, 'moderate': 1, 'quite rich':3, 'rich':2 }

df['Saving accounts'] = df['Saving accounts'].map(mapeamento)   

In [299]:
mapeamento = {'good': 0, 'bad': 1}

df['Risk'] = df['Risk'].map(mapeamento)

In [300]:
#mapeamento = {'female': 1, 'male': 0}

#df['Sex'] = df['Sex'].map(mapeamento)

#transfomar as variaveis em 1 ou 0 piora o resultado, independente se homem = 0 e mulher = 1 ou homem = 1 e mulher = 0

In [301]:
# mapeamento = {'own': 0, 'rent': 1,'free':2}

# df['Housing'] = df['Housing'].map(mapeamento)

#### One hot encoding

In [302]:
#agrupar algusn valores de Purpose em um clusetr só, esses valores tem um baixo número de instancias então agrupa-los em uma categoria só parece valido
df['Purpose'] = df['Purpose'].replace({'repairs': 'others', 'domestic appliances': 'others', 'vacation/others': 'others'})

#agrupar mais motivos em um cluster só
#df['Purpose'] = df['Purpose'].replace({'repairs': 'others', 'domestic appliances': 'others', 'vacation/others': 'others','education':'others'})

In [303]:
#df = df.drop(columns=['Purpose'])

In [304]:
df = pd.get_dummies(df) #essa função aplica o one hot encoding em todas as colunas categoricas

In [305]:
df

,Age,Job,Saving accounts,Checking account,Credit amount,Duration,Risk,Sex_female,Sex_male,Housing_free,Housing_own,Housing_rent,Purpose_business,Purpose_car,Purpose_education,Purpose_furniture/equipment,Purpose_others,Purpose_radio/TV
1,0.053571,2,0,1.000000,0.777550,0.647059,1,True,False,False,True,False,False,False,False,False,False,True
2,0.535714,1,0,1.666667,0.251773,0.117647,0,False,True,False,True,False,False,False,True,False,False,False
4,0.607143,2,0,0.000000,0.630115,0.294118,1,False,True,True,False,False,False,True,False,False,False,False
6,0.607143,2,3,0.666667,0.352564,0.294118,0,False,True,False,True,False,False,False,False,True,False,False
7,0.285714,3,0,1.000000,0.913530,0.470588,0,False,True,False,False,True,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.214286,1,0,0.333333,0.202673,0.117647,0,True,False,False,True,False,False,False,False,True,False,False
996,0.375000,3,0,0.000000,0.491953,0.382353,0,False,True,False,True,False,False,True,False,False,False,False
997,0.339286,2,0,0.333333,0.075559,0.117647,0,False,True,False,True,False,False,False,False,False,False,True
998,0.071429,2,0,0.000000,0.217540,0.602941,1,False,True,True,False,False,False,False,False,False,False,True


### Discretização

#### Agrupando categorias

In [306]:
from sklearn.cluster import KMeans
#aqui tentei utilizar kmeans para clusterizar essas variaveis mas nao melhorou os resultados

#kmeans = KMeans(n_clusters=5, random_state=42)
#df['cluster'] = kmeans.fit_predict(df[['Credit amount']])

In [307]:
#kmeans = KMeans(n_clusters=3, random_state=42)
#df['cluster_age'] = kmeans.fit_predict(df[['Age']])
# clusterizar a idade aumenta a taxa de acerto do modelo

In [308]:
kmeans = KMeans(n_clusters=5, random_state=42)
#df['cluster_duration'] = kmeans.fit_predict(df[['Duration']])
# clusterizar a duração esperada da divida
#df = df.drop(columns=['Age','Duration'])

In [309]:
#df = df.drop(columns=['Age','Duration','Credit amount'])

### Finalização

In [310]:
#usei smote-tomek para melhorar ainda mais o balanceamento, adicionando dados sinteticos e removendo ruido ao mesmo tempo
#a técnica consiste em usar smote para gerar dados sinteticos e tomek links para remover as instancias que estao muito proximas umas das outras

from imblearn.combine import SMOTETomek

X = df.drop(columns=['Risk'])
y = df['Risk']

# Aplica SMOTE
smote_tomek = SMOTETomek(random_state=42)
X_resampled, y_resampled = smote_tomek.fit_resample(X, y)

from collections import Counter
print("Distribuição após SMOTETomek:", Counter(y_resampled))



Distribuição após SMOTETomek: Counter({1: 466, 0: 466})


In [311]:

X_resampled.to_csv("../data/df_X.csv", index=False)
y_resampled.to_csv('../data/df_y.csv', index=False)